## 직방 원룸 매물 데이터 수집

In [1]:
import requests
import pandas as pd

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

### 1\. 동이름으로 위도 경도 구하기

In [54]:
addr = "논현동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.51264572143555, 127.0301513671875)

### 2\. 위도 경도로 geohash 알아내기

In [55]:
#install geohash2
#pip install geohash2

In [56]:
import geohash2

In [65]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=6)
geohash

'wydm6s'

### 3\. geohash로 매물 아이디 가져오기

In [66]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
response = requests.get(url)
datas = response.json()["items"]
# len(datas), datas[0]
ids = [data["item_id"] for data in datas]
len(ids), ids[:5]

(355, [40064174, 40157452, 40208024, 40225695, 40244448])

### 4\. 매물 아이디로 매물 정보 가져오기

In [67]:
# 1000개 넘어가면 나눠서 수집
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "item_ids": ids
}
response = requests.post(url, params)
response

<Response [200]>

In [68]:
datas = response.json()["items"]
df = pd.DataFrame(datas)
df.tail(3)

,item_id,section_type,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_new,addressOrigin,action,contract,address,is_zzim,address1,address2,address3,item_bm_type
352,40327097,None,https://ic.zigbang.com/ic/items/40327097/1.jpg,월세,월세,100,150,72.55,"{'m2': 72.55, 'p': '21.9'}","{'m2': 72.55, 'p': '21.9'}",...,True,"{'local1': '서울시', 'local2': '강남구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",단기,강남구 논현동,False,서울시 강남구 논현동,None,None,ZIGBANG
353,40163386,None,https://ic.zigbang.com/ic/items/40163386/1.jpg,월세,월세,100,65,33.06,"{'m2': 33.06, 'p': '10'}","{'m2': 33.06, 'p': '10'}",...,False,"{'local1': '서울시', 'local2': '강남구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",단기,강남구 논현동,False,서울시 강남구 논현동,None,None,ZIGBANG
354,40321546,None,https://ic.zigbang.com/ic/items/40321546/1.jpg,전세,전세,12000,0,42.98,"{'m2': 42.98, 'p': '13'}","{'m2': 42.98, 'p': '13'}",...,True,"{'local1': '서울시', 'local2': '강남구', 'local3': '...","{'isRead': False, 'readAt': None, 'isInquired'...",,강남구 논현동,False,서울시 강남구 논현동,None,None,ZIGBANG


In [69]:
# 필요한 컬럼만 필터링
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor",
           "address1", "manage_cost"]
filtered_column_df = df[columns]
filtered_column_df.tail(5)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
350,40305638,월세,2000,140,50.59,2,3,서울시 강남구 논현동,5
351,40142654,전세,16000,0,26.45,3,3,서울시 강남구 논현동,10
352,40327097,월세,100,150,72.55,3,3,서울시 강남구 논현동,10
353,40163386,월세,100,65,33.06,반지하,3,서울시 강남구 논현동,9
354,40321546,전세,12000,0,42.98,반지하,4,서울시 강남구 논현동,20


In [71]:
# 주소에 망원동이 있는 데이터만 필터링
result_df = filtered_column_df[filtered_column_df["address1"].str.contains("논현동")]
result_df = result_df.reset_index(drop=True)
result_df.tail(5)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
350,40305638,월세,2000,140,50.59,2,3,서울시 강남구 논현동,5
351,40142654,전세,16000,0,26.45,3,3,서울시 강남구 논현동,10
352,40327097,월세,100,150,72.55,3,3,서울시 강남구 논현동,10
353,40163386,월세,100,65,33.06,반지하,3,서울시 강남구 논현동,9
354,40321546,전세,12000,0,42.98,반지하,4,서울시 강남구 논현동,20
